In [1]:
# import subprocess
# subprocess.call(['sh', './train.sh'])

# Training

In [2]:
from itertools import chain
from s_clip_scripts.main import main
from s_clip_scripts.params import parse_args

fashion = True

if fashion:
    str_args = '''--model RN50 
            --pretrained openai 
            --train-data Fashion-ALL
            --label-ratio 0.1
            --val-data Fashion-ALL
            --keyword-path keywords/fashion/class-name.txt
            --lr 5e-5 
            --batch-size 64 
            --warmup 10 
            --epochs 10
            --precision amp 
            --method base 
            --seed 0 
    '''
else:
    str_args = '''--model RN50 
            --pretrained openai 
            --train-data RS-ALL
            --label-ratio 0.1
            --val-data RS-ALL
            --imagenet-val RSICD-CLS \
            --keyword-path keywords/RS/class-name.txt
            --lr 5e-5 
            --batch-size 64 
            --warmup 10 
            --epochs 25
            --zeroshot-frequency 5 
            --precision amp 
            --method base 
            --seed 0 
    '''
str_args = str_args.split('\n')
str_args = [s.strip() for s in str_args]
str_args = [s.split(' ') for s in str_args]
str_args = list(chain(*str_args))
str_args = [s for s in str_args if len(s) > 0]
args = parse_args(str_args)

In [3]:
%%time
main(args) # Calls the main.py function of S-CLIP

Making log path: 2023_10_12-09_47_28-data_Fashion-ALL-ratio_0.1-model_RN50-method_base-keyword_none-seed_0
Running with a single process. Device cuda:0.
Getting data...
Fashion-ALL
Fashion-ALL
Data got.
Start epoch 0
Done training.
Done evaluating.
Start epoch 1
Done training.
Done evaluating.
Start epoch 2
Done training.
Done evaluating.
Start epoch 3
Done training.
Done evaluating.
Start epoch 4
Done training.
Done evaluating.
Start epoch 5
Done training.
Done evaluating.
Start epoch 6
Done training.
Done evaluating.
Start epoch 7
Done training.
Done evaluating.
Start epoch 8
Done training.
Done evaluating.
Start epoch 9
Done training.
Done evaluating.
CPU times: user 3h 9min 19s, sys: 2min 26s, total: 3h 11min 46s
Wall time: 42min 11s


# Eval

In [3]:
%%time
checkpoint = '2023_10_12-09_47_28-data_Fashion-ALL-ratio_0.1-model_RN50-method_base-keyword_none-seed_0'
# checkpoint = '2023_10_11-10_26_36-data_RS-ALL-ratio_0.1-model_RN50-method_base-keyword_none-seed_0'
if 'Fashion' in checkpoint:
    zeroshot_datasets = ["Fashion200k-SUBCLS", "Fashion200k-CLS", "FashionGen-CLS", "FashionGen-SUBCLS", "Polyvore-CLS", ]
    retrieval_datasets = ["FashionGen", "Polyvore", "Fashion200k",]
else:
    zeroshot_datasets = ["RSICD-CLS", "UCM-CLS"] # "WHU-RS19", "RSSCN7", "AID" -> NOT WORKING bc of different data-loading workings
    retrieval_datasets = ["RSICD", "UCM", "Sydney"]

for dataset in zeroshot_datasets:
    str_args = ['--name', checkpoint, '--imagenet-val', dataset]
    args = parse_args(str_args)
    main(args)

for dataset in retrieval_datasets:
    str_args = ['--name', checkpoint, '--val-data', dataset]
    args = parse_args(str_args)
    main(args)

Running with a single process. Device cuda:0.
=> resuming checkpoint '2023_10_12-09_47_28-data_Fashion-ALL-ratio_0.1-model_RN50-method_base-keyword_none-seed_0/checkpoints/epoch_latest.pt' (epoch 10)
Getting data...
Fashion200k-SUBCLS
['blazers and suit jackets', 'blouses', 'cargo pants', 'casual and day dresses', 'casual jackets', 'cocktail dresses', 'cropped pants', 'denim jackets', 'full length pants', 'fur jackets', 'gowns', 'harem pants', 'knee length skirts', 'leather jackets', 'leggings', 'long sleeved tops', 'maxi and long dresses', 'maxi skirts', 'mid length skirts', 'mini and short dresses', 'mini skirts', 'padded and down jackets', 'prom and formal dresses', 'shirts', 'short sleeve tops', 'skinny pants', 'sleeveless and tank tops', 'straight-leg pants', 't-shirts', 'waistcoats and gilets', 'wide-leg and palazzo pants']
Data got.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29824/29824 [03:29<00:00, 142.36it/s]


Running with a single process. Device cuda:0.
=> resuming checkpoint '2023_10_12-09_47_28-data_Fashion-ALL-ratio_0.1-model_RN50-method_base-keyword_none-seed_0/checkpoints/epoch_latest.pt' (epoch 10)
Getting data...
Fashion200k-CLS
['dresses', 'jackets', 'pants', 'skirts', 'tops']
Data got.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29824/29824 [03:26<00:00, 144.42it/s]


Running with a single process. Device cuda:0.
=> resuming checkpoint '2023_10_12-09_47_28-data_Fashion-ALL-ratio_0.1-model_RN50-method_base-keyword_none-seed_0/checkpoints/epoch_latest.pt' (epoch 10)
Getting data...
FashionGen-CLS
['backpacks', 'bag accessories', 'belts & suspenders', 'blankets', 'boots', 'briefcases', 'clutches & pouches', 'dresses', 'duffle & top handle bags', 'duffle bags', 'espadrilles', 'eyewear', 'fine jewelry', 'flats', 'gloves', 'hats', 'heels', 'jackets & coats', 'jeans', 'jewelry', 'jumpsuits', 'keychains', 'lace ups', 'lingerie', 'loafers', 'messenger bags', 'messenger bags & satchels', 'pants', 'pocket squares & tie bars', 'pouches & document holders', 'sandals', 'scarves', 'shirts', 'shorts', 'shoulder bags', 'skirts', 'sneakers', 'socks', 'suits & blazers', 'sweaters', 'swimwear', 'ties', 'tops', 'tote bags', 'underwear & loungewear']
Data got.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:01<00:00, 63.16it/s]


Running with a single process. Device cuda:0.
=> resuming checkpoint '2023_10_12-09_47_28-data_Fashion-ALL-ratio_0.1-model_RN50-method_base-keyword_none-seed_0/checkpoints/epoch_latest.pt' (epoch 10)
Getting data...
FashionGen-SUBCLS
['backpacks', 'bag accessories', 'belts & suspenders', 'blankets', 'boots', 'briefcases', 'clutches & pouches', 'dresses', 'duffle & top handle bags', 'duffle bags', 'espadrilles', 'eyewear', 'fine jewelry', 'flats', 'gloves', 'hats', 'heels', 'jackets & coats', 'jeans', 'jewelry', 'jumpsuits', 'keychains', 'lace ups', 'lingerie', 'loafers', 'messenger bags', 'messenger bags & satchels', 'pants', 'pocket squares & tie bars', 'pouches & document holders', 'sandals', 'scarves', 'shirts', 'shorts', 'shoulder bags', 'skirts', 'sneakers', 'socks', 'suits & blazers', 'sweaters', 'swimwear', 'ties', 'tops', 'tote bags', 'underwear & loungewear']
Data got.


  0%|                                                                                                                                                               | 0/64 [00:00<?, ?it/s]


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/vol/tensusers5/nhollain/thesis_env/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/vol/tensusers5/nhollain/thesis_env/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/vol/tensusers5/nhollain/thesis_env/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/vol/tensusers5/nhollain/s_clip_scripts/data_loader.py", line 301, in __getitem__
    y = self.classes.index(cls_name)
ValueError: 'denim jackets' is not in list
